<!--HEADER-->
[*Notas sobre modelagem da epidemia de Covid-19*](https://github.com/rmsrosa/modcovid19) / [*IM-UFRJ*](https://www.im.ufrj.br).

<!--BADGES-->
<a href="../slides/16.00.Aula-Modelos_individuais_reformulacao.slides.html" target="_blank"><img align="left" src="https://img.shields.io/badge/local-slides-darkgreen" alt="localslides" title="Local Slides"></a>
&nbsp;

<!--NAVIGATOR-->
[<- Modelos individuais - velocidade de processsamento](15.00.Aula-Modelos_individuais_velocidade.ipynb) | [Página Inicial](00.00-Pagina_Inicial.ipynb) 

---


# Modelos individuais - reformulação da implementação

- Reformular o método para acelerar a execução, baseado nos testes da última aula.

**Importando bibliotecas e definindo funções a serem usadas abaixo**

In [1]:
import datetime # date and time tools

import os, sys

import numpy as np

from numba import njit, prange
import threading

import math
from timeit import repeat

import matplotlib.pyplot as plt
import seaborn as sns

import io, base64
from IPython.display import Image, HTML

In [2]:
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))
import episiming

In [3]:
dt_string = datetime.datetime.now().strftime("%d/%b/%Y")
print(f"Atualização mais recente do kernel: {dt_string}")

Atualização mais recente do kernel: 21/May/2020


In [4]:
sns.set_style("darkgrid")

## Modificações

### Passo vetorial

- Implementamos o "decorador" `numba.njit` na **função** `passo_vetorial` e em todas as funções que são chamadas por ela.

- A essa implementação demos o nome temporário de `passo_vetorial_jit`.

- Com ela, cada **cenário** foi acrescido do **método** `evolucao_jit`.

- Para usar o decorador `numba.njit`, tivemos que mudar os parâmetros
    - `redes` (lista de grafos do `networkx`)
    - `redes_tx_transmissao` (lista de `ndarray` do `numpy`)
    
- Foram transformados, dentro da parte de preparação função de evolução, antes de entrar no ciclo de evolução, para
    - `conexoes` (typed List de typed List do `numba`)
    - `tx_transmissao` (typed List de `ndarray` do `numpy`)

dados por
```python
conexoes = List()
for rede in redes:
    conexoes_aux = list(rede.edges)
    typed_conexoes = List()
    for c in conexoes_aux:
        typed_conexoes.append(c)
    conexoes.append(typed_conexoes)
    
tx_transmissao = List()
for rede_tx in redes_tx_transmissao:
    tx_transmissao.append(rede_tx)
```

### Evolução de cada simulação

Também usamos o decorador `@njit` no ciclo da simulação, criando uma função `simulacao`.

Para isso, mudamos, também, as linhas

```python
S = np.hstack([S, np.count_nonzero(populacao==1)])
I = np.hstack([I, np.count_nonzero(populacao==2)])
R = np.hstack([R, np.count_nonzero(populacao==3)])
```

por

```python
S = hstack_jit(S, np.count_nonzero(pop_estado==1))
I = hstack_jit(I, np.count_nonzero(pop_estado==2))
R = hstack_jit(R, np.count_nonzero(pop_estado==3))
```

onde

```python
@njit
def hstack_jit(vetor, valor):
    return np.append(vetor, valor)
```

## Comparação

Para fins de comparação, rodamos alguns cenários diferentes, com os seguintes dados de simulação:

In [5]:
num_sim = 10
dt = 1 # primeiro passo
t_0 = 0 # instante initial
num_dt = 60 # inst
dados_temporais = [t_0, dt, num_dt]

### Rede completa com 60 indivíduos

In [6]:
completo_60 = episiming.cenarios.RedeCompleta(60, 8, 0.5, 0.2)

In [7]:
aux = completo_60.evolui_jit(dados_temporais, num_sim)

In [8]:
%time X = completo_60.evolui(dados_temporais, num_sim)
%time X_jit = completo_60.evolui_jit(dados_temporais, num_sim)

CPU times: user 2.16 s, sys: 15.2 ms, total: 2.17 s
Wall time: 2.17 s
CPU times: user 222 ms, sys: 1.4 ms, total: 224 ms
Wall time: 229 ms


### Rede completa com 600 indivíduos

In [9]:
completo_600 = episiming.cenarios.RedeCompleta(600, 8, 0.5, 0.2)

In [10]:
%time X_j = completo_600.evolui(dados_temporais, num_sim)

CPU times: user 1min 52s, sys: 387 ms, total: 1min 52s
Wall time: 1min 54s


In [11]:
%time X_jit = completo_600.evolui_jit(dados_temporais, num_sim)

CPU times: user 15.9 s, sys: 73.8 ms, total: 16 s
Wall time: 16.2 s


### Rede Pop350

In [12]:
cenario_pop_350 = episiming.cenarios.Pop350()

In [13]:
%time X = cenario_pop_350.evolui(dados_temporais, num_sim)
%time X_jit = cenario_pop_350.evolui_jit(dados_temporais, num_sim)

CPU times: user 13 s, sys: 68.6 ms, total: 13.1 s
Wall time: 13.4 s
CPU times: user 5.18 s, sys: 22.8 ms, total: 5.2 s
Wall time: 5.24 s


<!--NAVIGATOR-->

---
[<- Modelos individuais - velocidade de processsamento](15.00.Aula-Modelos_individuais_velocidade.ipynb) | [Página Inicial](00.00-Pagina_Inicial.ipynb) 